# 1.1 Verificação de Qualidade dos Dados

**Objetivo:** Identificar problemas de qualidade (valores nulos, duplicatas, outliers, inconsistências)

## Importação de Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

sns.set_theme(style="whitegrid")
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

## Carregamento dos Dados

In [2]:
data_path = Path('../data/raw/Cardiovascular_Disease_Dataset.csv')
df = pd.read_csv(data_path)

print(f"Dataset: {df.shape[0]} linhas x {df.shape[1]} colunas")

Dataset: 1000 linhas x 14 colunas


## Visualização Inicial

In [3]:
df.head()

,patientid,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
0,103368,53,1,2,171,0,0,1,147,0,5.3,3,3,1
1,119250,40,1,0,94,229,0,1,115,0,3.7,1,1,0
2,119372,49,1,2,133,142,0,0,202,1,5.0,1,0,0
3,132514,43,1,0,138,295,1,1,153,0,3.2,2,2,1
4,146211,31,1,1,199,0,0,2,136,0,5.3,3,2,1


In [4]:
df.tail()

,patientid,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
995,9949544,48,1,2,139,349,0,2,183,1,5.6,2,2,1
996,9953423,47,1,3,143,258,1,1,98,1,5.7,1,0,0
997,9965859,69,1,0,156,434,1,0,196,0,1.4,3,1,1
998,9988507,45,1,1,186,417,0,1,117,1,5.9,3,2,1
999,9990855,25,1,0,158,270,0,0,143,1,4.7,0,0,0


## Informações do Dataset

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patientid          1000 non-null   int64  
 1   age                1000 non-null   int64  
 2   gender             1000 non-null   int64  
 3   chestpain          1000 non-null   int64  
 4   restingBP          1000 non-null   int64  
 5   serumcholestrol    1000 non-null   int64  
 6   fastingbloodsugar  1000 non-null   int64  
 7   restingrelectro    1000 non-null   int64  
 8   maxheartrate       1000 non-null   int64  
 9   exerciseangia      1000 non-null   int64  
 10  oldpeak            1000 non-null   float64
 11  slope              1000 non-null   int64  
 12  noofmajorvessels   1000 non-null   int64  
 13  target             1000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 109.5 KB


In [6]:
df.describe()

,patientid,age,gender,chestpain,restingBP,serumcholestrol,fastingbloodsugar,restingrelectro,maxheartrate,exerciseangia,oldpeak,slope,noofmajorvessels,target
count,1.000000e+03,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,5.048704e+06,49.24200,0.765000,0.980000,151.747000,311.447000,0.296000,0.748000,145.477000,0.498000,2.707700,1.540000,1.222000,0.580000
std,2.895905e+06,17.86473,0.424211,0.953157,29.965228,132.443801,0.456719,0.770123,34.190268,0.500246,1.720753,1.003697,0.977585,0.493805
min,1.033680e+05,20.00000,0.000000,0.000000,94.000000,0.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.536440e+06,34.00000,1.000000,0.000000,129.000000,235.750000,0.000000,0.000000,119.750000,0.000000,1.300000,1.000000,0.000000,0.000000
50%,4.952508e+06,49.00000,1.000000,1.000000,147.000000,318.000000,0.000000,1.000000,146.000000,0.000000,2.400000,2.000000,1.000000,1.000000
75%,7.681877e+06,64.25000,1.000000,2.000000,181.000000,404.250000,1.000000,1.000000,175.000000,1.000000,4.100000,2.000000,2.000000,1.000000
max,9.990855e+06,80.00000,1.000000,3.000000,200.000000,602.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,1.000000


### Verificação de valores zerados críticos

Analisamos o total de valores críticos em 'serumcholesterol' e 'slope'. Do ponto de vista clínico, é impossível apresentar nível de colesterol igual a 0. Já 'slope' é uma variável categórica que deveria conter apenas os valores 1, 2 e 3.

In [7]:
# Verificacao de Valores Zerados Criticos

zero_chol = (df['serumcholestrol'] == 0).sum()
pct_zero_chol = (zero_chol / len(df)) * 100

zero_slope = (df['slope'] == 0).sum()
pct_zero_slope = (zero_slope / len(df)) * 100

print(f"\nSerum Cholesterol (chol):")
print(f"  Total de zeros: {zero_chol} ({pct_zero_chol:.2f}%)")

print(f"\nSlope of ST segment (slope):")
print(f"  Total de zeros: {zero_slope} ({pct_zero_slope:.2f}%)")



Serum Cholesterol (chol):
  Total de zeros: 53 (5.30%)

Slope of ST segment (slope):
  Total de zeros: 180 (18.00%)


Analisamos os valores do 'target' para as instâncias onde 'slope' era 0, visando identificar qual categoria esse valor representaria, já que esse grupo (180 linhas) compõe uma parcela significativa (18%) dos dados.

In [8]:
# Mostrar linhas onde slope == 0 e os valores do target dessa linha
slope_zero = df[df['slope'] == 0]
if not slope_zero.empty:
    display(slope_zero[['slope', 'target']].reset_index(drop=True))
    print('\nContagem dos valores de target onde slope == 0:')
    print(slope_zero['target'].value_counts(dropna=False))
else:
    print('Nenhuma linha com slope == 0 encontrada.')

,slope,target
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
175,0,0
176,0,0
177,0,0
178,0,0



Contagem dos valores de target onde slope == 0:
target
0    180
Name: count, dtype: int64


## Valores Nulos

In [9]:
null_summary = pd.DataFrame({
    'Coluna': df.columns,
    'Qtd_Nulos': df.isnull().sum().values,
    'Pct_Nulos': (df.isnull().sum() / len(df) * 100).values.round(2)
})

print("Resumo de Valores Nulos:")
print(null_summary.to_string(index=False))
print(f"\nTotal de valores nulos: {df.isnull().sum().sum()}")

Resumo de Valores Nulos:
           Coluna  Qtd_Nulos  Pct_Nulos
        patientid          0        0.0
              age          0        0.0
           gender          0        0.0
        chestpain          0        0.0
        restingBP          0        0.0
  serumcholestrol          0        0.0
fastingbloodsugar          0        0.0
  restingrelectro          0        0.0
     maxheartrate          0        0.0
    exerciseangia          0        0.0
          oldpeak          0        0.0
            slope          0        0.0
 noofmajorvessels          0        0.0
           target          0        0.0

Total de valores nulos: 0


# Verificação de Duplicatas


In [10]:
duplicates = df.duplicated().sum()

if duplicates > 0:
    print(f"\nRED FLAG: {duplicates} linhas duplicadas encontradas")
    print("\nVisualizando primeiras duplicatas:")
    print(df[df.duplicated(keep=False)].head(10))
else:
    print("OK: Nenhuma duplicata encontrada")

OK: Nenhuma duplicata encontrada


# Deteccao de Outliers (IQR Method)


In [11]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.drop('patientid')

outlier_summary = []

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    n_outliers = len(outliers)
    pct_outliers = (n_outliers / len(df)) * 100
    
    outlier_summary.append({
        'Variavel': col,
        'N_Outliers': n_outliers,
        'Pct_Outliers': round(pct_outliers, 2),
        'Min': df[col].min(),
        'Max': df[col].max()
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df.to_string(index=False))

         Variavel  N_Outliers  Pct_Outliers  Min   Max
              age           0           0.0 20.0  80.0
           gender         235          23.5  0.0   1.0
        chestpain           0           0.0  0.0   3.0
        restingBP           0           0.0 94.0 200.0
  serumcholestrol           0           0.0  0.0 602.0
fastingbloodsugar           0           0.0  0.0   1.0
  restingrelectro           0           0.0  0.0   2.0
     maxheartrate           0           0.0 71.0 202.0
    exerciseangia           0           0.0  0.0   1.0
          oldpeak           0           0.0  0.0   6.2
            slope           0           0.0  0.0   3.0
 noofmajorvessels           0           0.0  0.0   3.0
           target           0           0.0  0.0   1.0


A partir dessa verificação, observamos que o principal problema reside nos valores inconsistentes de 'serumcholesterol' e 'slope'.